In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import datetime
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
from keras import layers
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate, Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, AveragePooling2D, BatchNormalization, Dropout
%matplotlib inline
import shutil
import scipy.io
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [3]:
print(os.getcwd())
for x in range(3):
    os.chdir("..")
print(os.getcwd())

/Users/hoangminhthao/Documents/GitHub/HeartResearch/Experiment/Approach/Own model approach
/Users/hoangminhthao/Documents/GitHub/HeartResearch


In [4]:
main_data_dir = os.getcwd() + "/Data set"
print(os.listdir(main_data_dir))

['.DS_Store', 'README.md', 'Label.csv', 'Output.png', 'TrainingSet3']


In [5]:
data_mat_files = glob(main_data_dir + "/TrainingSet*/*")
print(type(data_mat_files[0]))
print(data_mat_files[0])

<class 'str'>
/Users/hoangminhthao/Documents/GitHub/HeartResearch/Data set/TrainingSet3/A6442.mat


In [6]:
label_path = main_data_dir + "/Label.csv"
label_df = pd.read_csv(label_path)
label_df.head()

,Recording,First_label,Second_label,Third_label
0,A0001,5,NaN,NaN
1,A0002,1,NaN,NaN
2,A0003,2,NaN,NaN
3,A0004,2,NaN,NaN
4,A0005,7,NaN,NaN


In [15]:
print(np.unique(label_df['First_label'].values))

[1 2 3 4 5 6 7 8 9]


In [7]:
mat = scipy.io.loadmat(data_mat_files[0])
print(mat)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Dec 19 15:31:52 2017', '__version__': '1.0', '__globals__': [], 'ECG': array([[(array(['Female'], dtype='<U6'), array([[62]], dtype=uint8), array([[-0.02172756, -0.03226859, -0.02439572, ..., -0.04939739,
                -0.04311943, -0.07330249],
               [-0.06569242, -0.09713388, -0.08775478, ..., -0.09506198,
                -0.09508135, -0.10501402],
               [-0.04396486, -0.06486529, -0.06335906, ..., -0.04566459,
                -0.05196191, -0.03171153],
               ...,
               [-0.09784493, -0.13342814, -0.1202063 , ..., -0.10746858,
                -0.10505069, -0.10856059],
               [-0.07737291, -0.10661326, -0.1001836 , ..., -0.10885306,
                -0.10040558, -0.1077501 ],
               [-0.12013434, -0.17354123, -0.17562855, ..., -0.07852785,
                -0.07429978, -0.07726881]]))                                                                              

In [8]:
 for signal in data_mat_files:
        print(type(signal))
        break

<class 'str'>


In [9]:
label_df[label_df['Recording'] == 'A0005']['First_label'].values[0]

7

In [21]:
def data_extract(signal_raw_list, label_raw, min_len = 3000):
    signal_list = []
    label_list = []
    
    for signal in signal_raw_list:
        mat = scipy.io.loadmat(signal)
        signal_list.append(mat['ECG'][0][0][2][:,:min_len])
        
        signal_number = signal.split('/')[-1].split('.')[0]
        label_list.append(label_raw[label_raw['Recording'] == signal_number]['First_label'].values[0])
    
    le = preprocessing.LabelEncoder()
    le.fit(label_list)
    labels = le.transform(label_list)
    print(labels)
    labels = tf.keras.utils.to_categorical(labels)
    print(labels)
    return (np.array(signal_list), np.array(labels))

In [22]:
signal, label = data_extract(data_mat_files, label_df)

[7 5 4 ... 4 1 4]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
print(signal.shape)
print(label.shape)

(2407, 12, 3000)
(2407, 9)
